<a href="https://colab.research.google.com/github/victor048/RAGExemplo/blob/main/RAGSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_openai import OpenAIEmbeddings, ChatOpenAI
    from langchain_community.vectorstores import Chroma
    from langchain_community.document_loaders import PyPDFLoader
    from langchain.chains.question_answering import load_qa_chain
    import os

In [21]:
os.environ["OPEN_AI_KEY"]="..."

In [22]:
#Load dos modelos (Embeddings e LLM)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens=200)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
#Carregar o PDF

pdf_link = "xpto.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [ ]:
#Separar em Chunks (Pedações de documentos)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    lenght_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [ ]:
# Salvar no Vector DB- Chroma

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
db.persist()

In [ ]:
#Carregar DB

vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

#Load Retriever
retriever = vectordb.as_retriever(search_kwargs={"k":2})
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
def ask(question):
  context = retriever.get_relevant_documents(question)
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
  return answer

In [ ]:
user_question = input("User: ")
answer = ask(user_question)
print("Answer: ", answer)